In [10]:
# Suppress TensorFlow warnings and info messages
from silence_tensorflow import silence_tensorflow
silence_tensorflow()

import os
os.environ["OPENCV_LOG_LEVEL"]="SILENT"
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

# Available backend options are: "jax", "torch", "tensorflow".
os.environ["KERAS_BACKEND"] = "tensorflow"

import cv2
import time

import os
import json
import keras
import numpy as np
import matplotlib.pyplot as plt

@keras.saving.register_keras_serializable()
def scaling(x, scale=1.0):
	return x * scale

# load the facenet128 model
try:
	model = keras.saving.load_model("hf://logasja/FaceNet")
	if model:
		# preload model to boost performance
		test = model.predict(np.zeros((1, 160, 160, 3)))
except Exception as e:
    print("Failed to load FaceNet Model...")
    # print(e)


Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step


In [14]:
# Important Variables
# faces path / data train
FACE_PATH = "./img"

if not os.path.exists(FACE_PATH):
    print("path : ./img => CREATING...")
    os.makedirs(FACE_PATH, exist_ok=True)
    print("path : ./img => CREATED...")
else:
    print("path : ./img => OK...")

# Signatures / Embedded Faces
SIGNATURES_PATH = "./data/signatures.json"

if not os.path.exists(SIGNATURES_PATH):
    print("file : signatures.json => CREATING...")
    json.dump({}, open(SIGNATURES_PATH, mode="w"), indent=4)
    print("file : signatures.json => CREATED...")
else:
    print("file : signatures.json => OK...")

path : ./img => OK...
file : signatures.json => OK...


In [ ]:
# configure camera
def setCamera(index: int = 0, width: int = 1280, height: int = 720):
    cap = cv2.VideoCapture(index)
    # Set the camera resolution
    cap.set(cv2.CAP_PROP_FRAME_WIDTH, width)
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, height)
    return cap

# access camera
def getCamera(index: int = 1, max_try: int = 5):
    try:       
        while max_try:
            cap = setCamera(index)
            
            # return cap if camera available
            if cap.isOpened():
                print(f"Camera-{index} is available")
                return cap
            else:
                print(f"Camera-{index} is not available!")
                index += 1
                max_try -= 1
                
        # open default camera if no additional camera is available
        if max_try == 0: 
            print("Access default camera...")
            cap = setCamera(0)
            return cap
            
            
    except Exception as e:
        print(f"An error occurred while accessing the camera: {e}")
        exit()

In [4]:
# render rectangle arround detected faces
def drawRectangle(face: list, frame: list, label="Face Detected", distance=0.0):
    x, y ,width, height = face[0], face[1], face[2], face[3]
    
    # Calculate font size based on the width of the rectangle
    # 0.002 is scale factor that determine font size
    font_scale = width * 0.002
    font_size = round(max(0.4, min(0.8, font_scale)), 2)
    
    # Draw a rectangle around the detected face
    frame = cv2.rectangle(frame, (x, y), (x + width, y + height), (0, 255, 0), 1)
    frame = cv2.rectangle(frame, (x,y-40), (x+width, y), (0, 255, 0), -2)
    
    # Draw a label above the rectangle
    frame = cv2.putText(frame, label + ', ' + str(round(distance, 2)), (x+5, y-15), cv2.FONT_HERSHEY_SIMPLEX, font_size, (0, 0, 0), 1, cv2.LINE_AA)
    
    return frame

In [5]:
# face detection
def faceDetection(frame: list, mode:str = "all"):
    # load the Haar Cascade classifier
    HaarCascade = cv2.CascadeClassifier(cv2.samples.findFile(cv2.data.haarcascades + "haarcascade_frontalface_default.xml"))
    
    # convert BGR to grayscale
    faceGray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # detect face
    # faceMaps : numpy array -> store face coordinates
    faceMaps = HaarCascade.detectMultiScale(
        image=faceGray,
        scaleFactor=1.1,
        minNeighbors=10,
        minSize=(128, 128),
        flags=cv2.CASCADE_SCALE_IMAGE,
    )
    
    # return face coordinates
    if len(faceMaps) == 0:
        # debug
        # print("0 faces detected")
        
        # return empty array
        return np.array([])
    else:
        # debug
        # print(f"{len(faceMaps)} faces detected")
        
        # return face maps
        if mode == "single":
            return [faceMaps[0]]  # return only first face coordinates
        if mode == "all":
            return faceMaps

In [6]:
# capture and save face image
def saveFaceImage(label: str, face: list):
    saved_path = os.path.join("./img/", label)
    
    # check if path exist
    if not os.path.exists(saved_path):
        # print(f"Creating folder for : \"{label}\"")
        os.makedirs(saved_path)
    # else:
        # print(f"Folder for \"{label}\" already exists.")
    
    # counting the number of files in the directory
    file_counter = len(os.listdir(saved_path)) + 1
    
    # generate filename and join it with face path
    filename = os.path.join(saved_path, f"{label}_{file_counter}.jpg")
    
    # save face
    try:
        # face = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
        cv2.imwrite(filename, face)
        print(f"Face image saved as {filename}")
    
    except Exception as e:
        print(f"Gagal menyimpan gambar: {e}")

In [7]:
# face embedding
def faceEmbedding(label: str):
    # assign empty faces_data and signatures
    faces_data = []
    signatures = []
    
    # load signatures (embedded faces) file
    signatures_data = json.load(open(SIGNATURES_PATH, mode="r"))
    
    # prepare faces data
    faces_path = f"./img/{label}/"
    for face_file in os.listdir(faces_path):
        face = cv2.imread(os.path.join(faces_path, face_file))  # read face file
        face = cv2.resize(face, (160, 160))                     # resize face image
        face = (face - face.mean()) / face.std()                # normalization
        faces_data.append(face)                               # append to faces_data
        
    # convert faces_data to numpy array
    faces_data = np.array(faces_data)
    print(f"face \"{label}\" {faces_data.shape} : OK....")
    
    # generate face signature
    print("\"{label}\" face signature : GENERATING...")
    signature = model.predict(faces_data)
    
    # combine 50 embedding to robust the signature
    signature_mean = np.mean(signature, axis=0)
    signature_median = np.median(signature, axis=0)
    
    # convert all numpy arrays to lists for JSON serialization
    signatures.append(signature.tolist())               # [0] : raw signature
    signatures.append(signature_mean.tolist())          # [1] : mean signature
    signatures.append(signature_median.tolist())        # [2] : median signature

    # update data with new signature
    signatures_data.update(({label: signatures}))
    
    # add new signature to signatures.json
    json.dump(signatures_data, open(SIGNATURES_PATH, mode="w"), indent=4)
    print("\"{label}\" face signature : GENERATED...")

In [26]:
# face recognition
def faceRecognition(face, threshold: float = 7.0):
    # prepare face
    face = cv2.resize(face, (160, 160))         # Resize the face to 160x160 pixels
    face = (face - face.mean()) / face.std()    # Normalize the face
    face = np.expand_dims(face, axis=0)         # Expand dimensions to match model input shape
    face = model.predict(face)                  # generate signature
    
    # assign list for recognition
    recognition = []
    
    # load signatures.json
    signatures = json.load(open(SIGNATURES_PATH, mode="r"))
    
    # calculate distance
    for label, signature in signatures.items():
        distance = np.linalg.norm(signature[1] - face)
        recognition.append([label, distance])
        
    # find identity
    identity = (min(recognition))

    # set threshold
    if distance <= threshold:
        label, dist = identity[0], identity[1]
    else:
        label, dist = "unknown", identity[1]
    
    # return label and distance
    return label, dist

In [28]:
# Access camera
cap = getCamera()

DETECTION_MODE = "all"

while cap.isOpened():
    # assign the key and read every 1ms
    key = cv2.waitKey(1) & 0xFF
    
    # read the frame
    # ret   : boolean
    # frame : numpy array
    _, frame = cap.read()
    
    # face detection
    faceMaps = faceDetection(frame, mode=DETECTION_MODE)
    # debug
    # print(type(faceMaps), len(faceMaps),"\n", faceMaps)
    
    # process detected faces
    for face_region in faceMaps:
        # drawRectangle(face, frame)
        
        # extract the face region from the frame
        x, y, w, h = face_region[0], face_region[1], face_region[2], face_region[3]
        face = frame[y+1:y+h, x+1:x+w]
        
        # save face image when pressing enter
        # if key == ord('\r'):
        #     saveFaceImage(label="yasir", face=face)
        
        # face recognition
        label, distance = faceRecognition(face, threshold=10.0)
        drawRectangle(face=face_region, frame=frame, label=label, distance=distance)
        # debug
        # print(label, distance)
    
    # display log/information
    cv2.putText(frame, f"Detected Mode : {DETECTION_MODE}", (10, frame.shape[0] - 70), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1, cv2.LINE_AA)
    cv2.putText(frame, f"Detected Faces : {len(faceMaps)}", (10, frame.shape[0] - 50), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1, cv2.LINE_AA)
    cv2.putText(frame, f"First Face : {label, round(distance, 4)}", (10, frame.shape[0] - 30), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1, cv2.LINE_AA)
    cv2.putText(frame, f"Resolution : {frame.shape}",  (10, frame.shape[0] - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1, cv2.LINE_AA)
    
    # display camera streaming
    cv2.imshow("Face Recognition", frame)
    
    # reduce cpu usage
    time.sleep(0.01)
    
    # press q for exit the loop
    if key == ord("q"):
        break
    
cap.release()
cv2.destroyAllWindows()

[ WARN:0@846.536] global cap_v4l.cpp:913 open VIDEOIO(V4L2:/dev/video1): can't open camera by index
[ERROR:0@846.725] global obsensor_uvc_stream_channel.cpp:158 getStreamChannelGroup Camera index out of range
[ WARN:0@846.726] global cap_v4l.cpp:913 open VIDEOIO(V4L2:/dev/video2): can't open camera by index
[ERROR:0@846.728] global obsensor_uvc_stream_channel.cpp:158 getStreamChannelGroup Camera index out of range


Camera-1 is not available!
Camera-2 is not available!
Camera-3 is available
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━